# Estruturas Criptográficas - Criptografia e Segurança da Informação

[Grupo 03](https://paper.dropbox.com/doc/Estruturas-Criptograficas-2023-2024-Trabalhos-Praticos-8WcsdZARGLv0nXS9KasmK)

(PG54177) Ricardo Alves Oliveira 

(PG54236) Simão Oliveira Alvim Barroso

## TP2 - Exercício 2

2. Uma das aplicações mais importantes do teorema chinês dos restos (CRT) em criptografia é a transformada NTT “Number Theoretic Transform”.  Esta transformada é uma componente importantes de “standards” PQC  como o Kyber e o Dilithium mas também de outros algoritmos submetidos ao concurso NIST PQC.  
A transformação NTT tem várias opções e aquela que está apresentada no +Capítulo 4:  Problemas Difíceis  usa o CRT.
Neste problema pretende-se uma implementação Sagemath  do NTT-CRT tal como é descrito nesse documento.

## Resolução

Para resolver este exercício comecamos por instalar e importar o SageMath. 

In [ ]:
from sage.all import *
import random

A função find_q(n) tem como objetivo encontrar um número primo que satisfaça a condição q≡1mod(2N), onde N pertence a [32, 64, 128, 256, 512, 1024, 2048].

In [ ]:
def find_q(n):
    if not  n in [32,64,128,256,512,1024,2048]:
        raise ValueError("improper argument ",n)
        
    q = 1 + 2*n
    while True:
        if (q).is_prime():
            return q
        q += 2*n
    

De seguida implementamos a função NTT que recebe como argumentos:

- f: Polinómio de entrada
- N: Tamanho do polinómio
- xi: Raiz primitiva de N
- F: Campo finito onde serão realizadas as operações

Esta função devolve o polinómio transformado de acordo com a transformada NTT. Para tal foram implementadas as seguintes funções auxiliares:

- \_expand\_: Esta função recebe um polinómio e um tamanho N e devolve um polinómio de tamanho N com os coeficientes do polinómio de entrada e os restantes coeficientes a 0.
- \_ntt\_: Esta função recebe um polinómio de entrada e devolve o polinómio transformado de acordo com a transformada NTT.

Por fim, a implementação da transformada segue o algoritmo descrito no [Capítulo 4](https://paper.dropbox.com/doc/Estruturas-Criptograficas-2023-2024-SUMARIO-UoGN1qanDMVYkNV8D5avY).

In [ ]:
def ntt(f,N,xi,F):                            
    def _expand_(f): 
        u = f.list()
        return u + [0]*(N-len(u)) 
    
    def _ntt_(xi,N,f):
        if N==1:
            return f
        N_ = N/2 ; xi2 =  xi^2  
        f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
        ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  

        s  = xi ; ff = [F(0) for i in range(N)] 
        for i in range(N_):
            a = ff0[i] ; b = s*ff1[i]  
            ff[i] = a + b ; ff[i + N_] = a - b 
            s = s * xi2                     
        return ff 
    
    return _ntt_(xi,N,_expand_(f))

Para o cálculo da inversa da transformada NTT foi implementada a função ntt_inv que ao receber o polinómio transformado, o tamanho do polinómio e o array de bases do crt, devolve o polinómio original.

In [ ]:
def ntt_inv(ff,N,base):                             
    return sum([ff[i]*base[i] for i in range(N)])

A função auxiliar random_pol e responsável por gerar um polinómio aleatório a partir de um anel polinomial R. 

In [ ]:
def random_pol(R,args=None):
    return R.random_element(args)

Para testar a implementação precisamos agora de obter o N desejado e gerar o q correspondente, com recurso à função find_q(n) mencionada anteriormente. 

In [ ]:
N = int(input("Enter N: "))
q = find_q(N)
print("q = ",q)


Com as variáveis de entrada defenidas podemos agora definir:

- O campo finito F
- O anel polinomial R com base no campo finito F
- O gerador do anel de polinômios R, representado por w
- O polinómio g, utilizado para encontrar as raízes primitivas
- As raízes primitivas xi
- As raízes rs que são geradas segundo a forma `xi^(2i+1)`
- A base do CRT calculada através da função `crt_basis`

In [ ]:
F = FiniteField(q)   
R = PolynomialRing(F, name="w")
w = R.gen()
g = (w^N + 1)
xi = g.roots(multiplicities=False)[-1]
rs = [xi^(2*i+1) for i in range(N)] 
base = crt_basis([(w - r) for r in rs]) 

Com as variáveis definidas podemos agora testar a implementação da transformada NTT e da sua inversa. Para tal, geramos um polinómio aleatório de tamanho entre 1 e N-1.

In [ ]:
f = random_pol(R,N-random.randint(1,N-1))
print("f = ",end='')
for i in range(N-1):
    print(f"{f[i]}*w^{i} + ",end='')
print(f"{f[N-1]}*w^{N-1}")

De seguida calculamos o polinómio transformado através da função NTT.

In [ ]:
ff = ntt(f,N,xi,F)
print("ff = ",end='')
for i in range(N-1):
    print(f"{ff[i]}*w^{i} + ",end='')
print(f"{ff[N-1]}*w^{N-1}")

Por fim calculamos o polinómio original através da função ntt_inv, utilizando para isso o polinómio transformado, o tamanho do polinómio e o array de bases do crt.

Para verificar fazemos a comparação entre o polinómio original e o polinómio obtido através da inversa da transformada NTT do polinómio transformado.

In [ ]:

fff = ntt_inv(ff,N,base)
print("fff = ",end='')
for i in range(N-1):
    print(f"{fff[i]}*w^{i} + ",end='')
print(f"{fff[N-1]}*w^{N-1}")

print("Correto ? ",f == fff) 